# A Gibbs Sampler for Spam Detection

In an [earlier post](https://nbviewer.jupyter.org/github/bobflagg/gibbs-sampling-for-the-uninitiated/blob/master/Gibbs-sampling-for-the-Uninitiated.ipynb) I gave a Python implementation of the Gibbs sampler for text classification described in the excellent tutorial paper
[Gibbs Sampling for the Uninitiated](https://www.umiacs.umd.edu/~resnik/pubs/LAMP-TR-153.pdf).  In this notebook, I'll show how to use that sampler to detect spam.  


## A Spam-vs-Ham Training Corpus

I'll use a combination of the [Enron-Spam](http://www.aueb.gr/users/ion/data/enron-spam/) data set and the [SpamAssassin public corpus](https://spamassassin.apache.org/publiccorpus/) to buld a training set for our spam detector. To simpify the presentation, I've done some minimal pre-processing of the original data and collected 
the results in a file in which each line contains the class (ham or spam) and the text of an 
e-mail message separated by a tab.  The corpus is available in the data directory of the github repository [Gibbs Sampling for the Uninitiated](https://github.com/bobflagg/gibbs-sampling-for-the-uninitiated).  Once the archive containing the data is expanded, the text and labels can be loaded with the following method.

In [216]:
def read_data(path='data/spam-or-ham.txt'):
    fp = open(path, 'r')
    texts = []
    labels = []
    for line in fp:
        line = line.strip()
        if line:
            label, text = line.split('\t')
            labels.append(label)
            texts.append(text)
    fp.close()    
    return texts, labels
texts, labels = read_data()

Let's look at a few records:

In [211]:
for i in range(5): 
    print("Message %d [%s]:\n%s\n" % (i, labels[i], texts[i]))

Message 0 [ham]:
Louise - OEC will come along with EECC & NEPCO . Brian Stanley and Keith Dodson will run that group . Speak with you soon . Hope you are feeling well . Regards - Dan Louise Kitchen @ ECT 03/14/2001 12:18 PM To : Dan Leff/HOU/EES @ EES cc : Subject : OEC What happens to the OEC group - does that currently report to you and will it going forward or does that move too ?

Message 1 [spam]:
-- -- 1305094201867723127 Content-Type : text/plain ; charset= '' iso-6434-4 '' Content-Transfer-Encoding : quoted-printable Content-Description : continued ross terramycin Hi , Make your ordinary 56k modem go speeds of upto 250k+ ! ( Average increased speeds of 195 - 200K ! ) Download music/programs in seconds , not minutes ! This hardware is compatible with EVERY Dialup ISP ! Check it out : http : //click4point.com/ Do n't forget to save our site URL ! ( If its not up , try again later ! ) Goodbye , http : //click4point.com/r/ Jenna=20 -- -- 1305094201867723127 --

Message 2 [ham]:
Att

To select a vocabulary, I'll restrict to the 10,000 most frequently occurring lower case words in the corpus afer 
removing words that appear too frequently to be informative.

In [215]:
from collections import Counter
def select_vocabulary(texts, V, max_cnt=10000):
    counter = Counter()
    for text in texts:
        for word in text.split():
            counter[word.lower()] += 1    
    words = [w for w in counter.keys() if counter[w] < max_cnt]
    words = sorted(words, key=lambda x: counter[x])
    return set(words[-V:])

V = 10000
vocabulary = select_vocabulary(texts, V)
word2id = {w:i for i, w in enumerate(vocabulary)}
id2word = {i:w  for i, w in enumerate(vocabulary)}

Now I'll build a corpus that can be used in our sampler.

In [217]:
def build_corpus(texts, vocabulary):
    corpus = []
    for text in texts:
        words = [w.lower() for w in text.split() if w.lower() in vocabulary]
        ids = [word2id[w] for w in words]
        counter = Counter(ids)
        document = {(i,c) for i, c in counter.items()}
        corpus.append(document)
    return corpus

corpus = build_corpus(texts, vocabulary)

## Spam-or-Ham with Gibbs Sampling

The next few cells are taken from the 
[earlier post](https://nbviewer.jupyter.org/github/bobflagg/gibbs-sampling-for-the-uninitiated/blob/master/Gibbs-sampling-for-the-Uninitiated.ipynb)
and define the sampler and a utility method to evaluate results of the classification.

In [126]:
def sample_labels(J, gamma_pi):
    pi = beta(gamma_pi[0], gamma_pi[1])
    return binomial(1, pi, J)

def initialize(W, labels, gamma_pi, gamma_theta):
    N = len(W)
    M = len(labels)
    V = len(gamma_theta)

    L = sample_labels(N - M, gamma_pi)
    theta = dirichlet(gamma_theta, 2)

    C = np.zeros((2,))
    C += gamma_pi
    cnts = np.zeros((2, V))
    cnts += gamma_theta
    
    for d, l in zip(W, labels.tolist() + L.tolist()):
        for i, c in d: cnts[l][i] += c
        C[l] += 1

    return {'C':C, 'N':cnts, 'L':L, 'theta':theta}

In [184]:
def update(state, X):
    C = state['C']
    N = state['N']
    L = state['L']
    theta = state['theta']
    # Update the labels for all documents:
    for j, l in enumerate(L):
        # Drop document j from the corpus:
        for i, c in X[j]: N[l][i] -= c
        C[l] -= 1  
        # Compute the conditional probability that L[j] = 1:  
        if C[0] == 1: pi = 1.0
        elif C[1] == 1 <= 0: pi = 0.0 
        else:
            # compute the product of probabilities (sum of logs)
            d = np.sum(C) - 1
            v0 = np.log((C[0] - 1.0) / d)
            v1 = np.log((C[1] - 1.0) / d)
            for i, c in X[j]:
                v0 += c * np.log(theta[0,i])
                v1 += c * np.log(theta[1,i])
            m = max(v0, v1)
            v0 = np.exp(v0 - m)
            v1 = np.exp(v1 - m)
            pi = v1 / (v0 + v1)
        # Sample the new label from the conditional probability:
        l = binomial(1, pi)
        L[j] = l
        # Add document j back into the corpus:
        C[l] += 1
        for i, c in X[j]: N[l][i] += c
    # Update the topics:
    theta[0] = dirichlet(N[0])
    theta[1] = dirichlet(N[1])

In [128]:
def run_sampler(W, labels, iterations, gamma_pi, gamma_theta):
    state = initialize(W, labels, gamma_pi, gamma_theta)
    X = W[len(labels):]
    for t in range(iterations): update(state, X)
    return state['L']

In [129]:
def compute_accuracy(L_true, L_predicted):
    correct = 0
    for i, l in enumerate(L_predicted):
        if L_true[i] == l: correct += 1
    accuracy = float(correct)/len(L_predicted)
    return accuracy

I'll run the sampler on the first 10,000 records in the corpus, training on 8,000 and testing on the remaining 2,000. Since the Gibbs sampler implementation has not yet been optimized, this takes about a minute on my machine.  Feel free to lower the value of N and the number of iterations when calling the following method.

In [237]:
def predict_spam_or_ham(N, p, iterations=100):
    gamma_pi = (1, 1)
    gamma_theta = [1] * V

    W = corpus[:N]
    n = int(N * p)
    labels_observed = np.array([0 if x == 'ham' else 1 for x in labels[:n]])
    labels_unobserved = np.array([0 if x == 'ham' else 1 for x in labels[n:N]])

    
    L = run_sampler(W, labels_observed, iterations, gamma_pi, gamma_theta)
    accuracy = compute_accuracy(labels_unobserved, L)
    return accuracy

In [238]:
%time accuracy = predict_spam_or_ham(N=10000, p=0.8, iterations=100)
print(accuracy)

CPU times: user 53.4 s, sys: 16 ms, total: 53.4 s
Wall time: 53.5 s
0.943


This gives an accuracy of about 94%, which we could certainly improve by a more careful selection of the vocabulary.

## Spam-or-Ham with scikit-learn

As a sanity check on the work above, I'll train and evaluate a Naive Bayes classifier on our data, using scikit-learn.

In [234]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

N = 10000
#N = len(texts)
p = 0.8
n = int(N * p)
X_train = texts[:n]
Y_train = labels[:n]
X_test = texts[n:N]
Y_test = labels[n:N]


pipeline = Pipeline([
    ('vectorizer',  CountVectorizer(vocabulary=vocabulary)),
    ('classifier',  MultinomialNB()) 
])
%time pipeline.fit(X_train, Y_train)
Y_predict = pipeline.predict(X_test)
accuracy = compute_accuracy(Y_test, Y_predict)
print(accuracy)

CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.45 s
0.96


This gives an accuracy of about 96%, which is definitely better than what I was able to achieve with the Gibbs sampler
but still in the same ball park.